<a href="https://colab.research.google.com/github/bhanuchaddha/The-Ai-Handbook/blob/main/4-AI-Agents/automatic_event_planner/Multi_Agent_Automatice_Event_Planner_using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automate Event Planner

In [ ]:
pip install crewai crewai-tools


In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
os.environ['SERPER_API_KEY']=userdata.get('SERPER_API_KEY')

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [ ]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [ ]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [ ]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [ ]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [ ]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    agent=logistics_manager
)

In [ ]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew


In [ ]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

In [ ]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Copenhagen",
    'tentative_date': "2025-02-15",
    'expected_participants': 100,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in Copenhagen that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Thought: I need to search for a suitable venue in Copenhagen that meets the requirements for a Tech Innovation Conference. I'll start by looking for venues specifically catering to tech conferences.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference venues in Copenhagen\"}"
## Tool Output: 

Search results: Title: Innovation Roundtable® Summit 2024 - Copenhagen, 5-7 November
Link: https://innovationroundtable.com/summit/innovation-2024/
Snippet: The Summit takes place at Tivoli Hotel & Congress Center, an award-winning venue inspired by the famous Tivoli Gardens. Located by Copenhagen's waterside, it is ...
---
Title: Top 10 Must-Attend Tech Meetups and Conferences in Denmark
Link: https://www.nucamp.co/blog/coding-bootcamp-denmark-dnk-top-10-mustattend-tech-meetups-and-conferences-in-denmark


In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Gdanskgade 2, 2150 Copenhagen, Denmark',
 'booking_status': 'Available for tech events and conferences',
 'capacity': 150,
 'name': 'CodeNode'}


In [ ]:
from IPython.display import Markdown
Markdown("marketing_report.md")

```markdown
# TechConnect World 2023 Marketing Plan

## Event Overview
- **Event Name:** TechConnect World 2023
- **Date:** June 19-21, 2023
- **Location:** Washington, DC, Gaylord National Harbor
- **Website:** [TechConnect World 2023](https://www.techconnectworld.com/World2023/)

## Conference Highlights
- **Key Features:**
  - Over 250 exhibitors and 2000 innovators participating.
  - Keynote speakers from industry-leading organizations and academia.
  - Multiple symposia on advanced materials, AI innovation, and sustainability.
  
- **Notable Speakers:**
  - Joseph Sopcisak, Office of the Under Secretary of Defense
  - Michael Arnold, University of Wisconsin
  - Tim Barckholtz, ExxonMobil
  - Jeannine M. Coburn, Worcester Polytechnic Institute, among others.

## Target Audience
- Industry professionals, investors, and academics in the fields of technology, innovation, and engineering.
- Startups and established companies looking to commercialize innovative technologies.

## Marketing Activities
1. **Email Campaign:**
   - Send targeted emails to previous attendees, industry stakeholders, and academic institutions.
   - Highlight marquee speakers and symposia relevant to various sectors.

2. **Social Media Promotion:**
   - Use platforms like LinkedIn, Twitter, and Facebook to create buzz around the event.
   - Share speaker highlights, session previews, and attendee testimonials from past events.

3. **Partnerships:**
   - Collaborate with tech communities, universities, and innovation hubs to promote the event.
   - Encourage partners to share the event through their channels.

4. **Content Marketing:**
   - Create blog posts and articles discussing key trends in technology relevant to the conference themes.
   - Feature interviews with speakers and innovators participating in the event.

5. **Webinars:**
   - Host pre-event webinars with selected speakers to discuss topics that will be covered during the conference.
   - Use these webinars to generate interest and encourage registrations.

6. **Promotional Materials:**
   - Develop eye-catching graphics and promotional videos to circulate online.
   - Create a landing page for the conference on your organization’s website.

## Engagement Strategies
- **Incentives for Early Registration:**
  - Offer discounts or exclusive content access for participants who register early.
  
- **Onsite Networking Opportunities:**
  - Organize networking sessions, meet-and-greet with speakers, and exhibitor showcases to drive interaction.
  
- **Feedback and Follow-up:**
  - Conduct surveys post-event to gather feedback, improve future events, and engage attendees with continued updates.

## Attendee Engagement Goals
- Aim to secure at least **100 confirmed attendees** by leveraging targeted outreach and clear communication of the event's unique value propositions.
- Engage attendees before, during, and after the event, ensuring sustained interest and community building.

---

Let’s harness the potential of TechConnect World to connect innovators, researchers, and investors, and drive forward the conversation in technology and innovation.
```